# MATH2504 Project 2 2022 Semester 2 Submission

[Assignment Instructions](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/project2.html)

Student names: Limao Chang, Tiarne Graves

In [ ]:
using Accessors, LinearAlgebra, StatsBase, Plots, Random
include("src/GeneralizedJacksonSim.jl")
using .GeneralizedJacksonSim
include("test/scenarios.jl")

: 

## Project Structure

In the root folder is `simulation_script.jl`, which imports and loads the `GeneralizedJacksonSim` module, as well as some other packages that are useful when working with `GeneralizedJacksonSim`.

The `src/` folder contains the source code for `GeneralizedJacksonSim`.

The `test/` folder contains tests which can be run in `test/runtests.jl`, as well as the four provided test scenarios, which can be found in `test/scenarios.jl`.

## Perspective Seminar

Due to the low quality of audio, this paragraph is in response to Dr Foeglein’s 2021 seminar: <br>
Dr Anna Foeglein knew that she wanted to be involved in simulation since she was a child, her interest being piqued when her mother brought home a computer. Anna studied a maths degree, majoring in pure mathematics and making some substantial contributions in her thesis that are still used today. Anna worked at the university for a while however was less interested in the tutoring aspect than she was in the software used to input and calculate students’ grades. She worked at simulation group for around 10 years before moving to the Integrated Logistics Company and AnyLogic where she currently works. She spends approximately 20% of her time selling software with AnyLogic, and the other 80% is spent working with the coal industry running simulations and models to organise coal shipment and delivery. <br>
There were many tools Dr Foeglein mentioned throughout her seminar including more code-based tools such as Simpy and SLX as well as more graphically based tools such as Jamsim. Even tools such as MATLAB Simulink are used, and which I found interesting given my current experience of MATLAB in mathematics courses at UQ. <br>
An interesting tip that I picked up from Anna’s talks are key things to form a good mathematical question, these things include being linked to a decision as well as an organisations goals, and being bounded, specific, and quantifiable. This links strongly to the concept of ‘mathematising’ that Anna introduced, an intriguing concept that means translating a regular question into one that can be mathematically answered.


## Task 1

**Features of amusement park that are not captured by model**:
- Amusement park attractions usually serve multiple people simultaneously, whereas the model assumes jobs can only be serviced one at a time
- People may cut in line and skip queues, so in reality it is not really a queue in the first-in, first-out (FIFO) sense

**Model assumptions that are unrealistic for the application of a theme park**:
- Assuming that queues (and by extension the theme park) have infinite capacity
- Assuming that people will unconditionally stay in a queue until they get to the front of the line - they may leave because the queue is too long, they want to go to another attraction, etc.
- Assuming that the time taken to move between attractions or leave the theme park is instant, as travelling between attractions takes time
- Service duration is independent from all other services - if people are at the theme park together as a party (e.g. a family travelling around the theme park), their service durations will not be independent from each other
- Arrival of jobs to nodes is exponentially distributed - the arrival rate will likely be higher during the day (more traffic) than in the evening, for example.

**If you were actually using this model to assess congestion at the park, would it be useful or not?**
While some model assumptions are unrealistic for application to a theme park, this model would still be a good starting point for congestion estimation. If a theme park can withstand the congestion according to this model it would be able to withstand realistic conjestion as many of the model assumptions that are unrealistic put increased strain on the system.

## Task 2

The total steady state mean queue lengths are plotted against $\rho$* below for the four scenarios.

In [ ]:
"""
    plot_total_ss_mean_queue_length(net::NetworkParameters)

Plots total steady state mean queue length as a function of ρ* for the given scenario.
"""
function plot_total_ss_mean_queue_length(net::NetworkParameters, scenario_number::Int64)
    ρ_star_values = 0.1:0.01:0.9
    total_ss_mean_queue_lengths = zeros(length(ρ_star_values))

    for (index, ρ_star) in enumerate(ρ_star_values)
        # adjust network parameters
        adjusted_scenario = set_scenario(net, ρ_star)
        ρ = compute_ρ(adjusted_scenario)

        total_ss_mean_queue_lengths[index] = sum(ρ ./ (1 .- ρ))
    end

    return plot(ρ_star_values,
                total_ss_mean_queue_lengths,
                xlabel="ρ",
                ylabel="Total Steady State Mean Queue Length",
                title="Scenario $scenario_number Theoretical")
end

p1 = plot_total_ss_mean_queue_length(scenario1, 1)
p2 = plot_total_ss_mean_queue_length(scenario2, 2)
p3 = plot_total_ss_mean_queue_length(scenario3, 3)
p4 = plot_total_ss_mean_queue_length(scenario4, 4)
        
plot(p1, p2, p3, p4, layout=(2, 2), legend=false, size=(800, 800))

: 

The below figure is return:

<center>
    <img src="img/task2.png" width="640" />
</center>

## Task 3

**Simulation engine**: `src/GeneralizedJacksonSim.jl` contains the `GeneralizedJacksonSim` module, as well as the simulation engine used (`sim_net()` below).

```julia
# src/GeneralizedJacksonSim.jl
"""
A discrete event simulation engine for Open Generalized Jackson Networks.
"""
module GeneralizedJacksonSim

import Base: isless
using Accessors, DataStructures, Distributions, StatsBase, Parameters, LinearAlgebra,
    Random, Plots

include("network_parameters.jl")
include("state.jl")
include("event.jl")

export NetworkParameters, compute_ρ, maximal_alpha_scaling, set_scenario, sim_net

"""
Runs a discrete event simulation of an Open Generalized Jackson Network `net`.

The simulation runs from time `0` to `max_time`.

Statistics about the total mean queue lengths are recorded from `warm_up_time` onwards
and the estimated value is returned.

This simulation does NOT keep individual customers' state, it only keeps the state which is
the number of items in each of the nodes.
"""
function sim_net(net::NetworkParameters;
                 max_time=10^6, warm_up_time=10^4, seed::Int64=42)::Float64
    Random.seed!(seed)

    # create priority queue and add standard events
    priority_queue = BinaryMinHeap{TimedEvent}()
    for q in 1:net.L
        push!(priority_queue, TimedEvent(ExternalArrivalEvent(q), 0.0))
    end
    push!(priority_queue, TimedEvent(EndSimEvent(), max_time))

    # initialise state and time
    state = QueueNetworkState(zeros(Int64, net.L), net.L, net)
    time = 0.0

    # set up queues integral for computing total mean queue length
    queues_integral = zeros(net.L)
    last_time = 0.0

    """
    Records the queue integral of the given state at the given point in time.
    """
    function record_integral(time::Float64, state::State)
        (time >= warm_up_time) && (queues_integral += state.queues * (time - last_time))
        last_time = time
    end

    record_integral(time, state)

    # simulation loop
    while true
        # process the next upcoming event
        timed_event = pop!(priority_queue)
        time = timed_event.time
        new_timed_events = process_event(time, state, timed_event.event)

        isa(timed_event.event, EndSimEvent) && break

        # add new spawned events to queue
        for nte in new_timed_events
            push!(priority_queue, nte)
        end

        # record mean queue length
        record_integral(time, state)
    end

    return sum(queues_integral / max_time)
end

end  # end of module
```


The other functionalities of the simulation engine (`NetworkParameters`, `State`s, `Event`s, and the functions related to each of them) have been separated into their own files, to keep the code nice and structured. Below are some snippets from each file:

```julia
# src/network_parameters.jl
"""
    NetworkParameters

# Fields
- `L::Int`: the dimension of the network (number of nodes)
- `α_vector::Vector{Float64}`: the external arrival rates α_i >= 0
- `μ_vector::Vector{Float64}`: the service rates μ_i > 0
- `P::Matrix{Float64}`: the L×L routing matrix P
- `c_s::Float64`: squared coefficient of variation of the service processes, defaults to 1.0
"""
@with_kw struct NetworkParameters
    L::Int
    α_vector::Vector{Float64}
    μ_vector::Vector{Float64}
    P::Matrix{Float64}
    c_s::Float64 = 1.0
end

# src/state.jl
"""
    QueueNetworkState

# Fields
- `queues::Vector{Int}`: vector of number of customers in each queue
- `num_queues::Int`: number of queues, equivalent to `length(queues)`
- `net::NetworkParameters`: the parameters of the network
"""
mutable struct QueueNetworkState <: State
    queues::Vector{Int}
    num_queues::Int
    net::NetworkParameters
end

"""
    next_arrival_time(s::State, q::Int)

Generates the next arrival time for the `q`th server. The duration of time between
external arrivals is exponentially distributed with mean 1 / s.net.α_vector[q].
"""
next_arrival_time(s::State, q::Int) = rand(Exponential(1/s.net.α_vector[q]))

"""
    next_service_time(s::State, q::Int)

Generates the next service time for the `q`th server. The service duration is gamma
distributed.
"""
next_service_time(s::State, q::Int) = rand(rate_scv_gamma(s.net.μ_vector[q], s.net.c_s))

# src/event.jl
"""
    process_event(time::Float64, state::State, event::ExternalArrivalEvent)

Process an arrival event from outside the system, and spawns a list of events that occur as
a consequence of this arrival.

On arrival, if the server is free (no jobs in the buffer/queue), the job starts to receive
service. If the server is busy, the job queues for service and waits for its turn.

The time between external arrival events for a given server is exponentially distributed,
and the service duration is gamma distributed.
"""
function process_event(time::Float64, state::State, event::ExternalArrivalEvent)
    q = event.q
    state.queues[q] += 1  # add to queue
    new_timed_events = TimedEvent[]

    # prepare next external arrival for this particular server
    push!(new_timed_events,
        TimedEvent(ExternalArrivalEvent(q), time + next_arrival_time(state, q)))

    # start serving this job if it is the only one in the queue
    if state.queues[q] == 1
        push!(new_timed_events,
            TimedEvent(EndOfServiceEvent(q), time + next_service_time(state, q)))
    end
    return new_timed_events
end

"""
    process_event(time::Float64, state::State, event::EndOfServiceEvent)

Process an end-of-service event, and spawns a list of events that occur as a consequence of
this end of service.

When a job completes service at a buffer, it either leaves the system, or moves to another
buffer (both happen immediately). After completing service in server i, a job moves to
server j with probability P[i, j], where P is the routing matrix.
"""
function process_event(time::Float64, state::State, event::EndOfServiceEvent)
    q = event.q
    state.queues[q] -= 1  # remove from queue
    @assert state.queues[q] >= 0
    new_timed_events = TimedEvent[]

    # if there is another customer in the queue, start serving them
    if state.queues[q] > 0
        service_time = next_service_time(state, q)
        push!(new_timed_events, TimedEvent(EndOfServiceEvent(q), time + service_time))
    end

    # simulate the next location for this job; indices 1:L are the probabilities of moving
    # to another server in the system, and the last index is the probability of exiting
    # the system
    L = state.net.L
    next_loc_weights = state.net.P[q, :]
    push!(next_loc_weights, 1 - sum(next_loc_weights))
    @assert sum(next_loc_weights) == 1
    next_loc = sample(1:L+1, Weights(next_loc_weights))

    if next_loc <= L
        # job is staying in the system
        state.queues[next_loc] += 1

        # start serving job if it is the only one in the queue
        if state.queues[next_loc] == 1
            service_time = next_service_time(state, next_loc)
            push!(new_timed_events,
                TimedEvent(EndOfServiceEvent(next_loc), time + service_time))
        end
    end
    return new_timed_events
end
```

**Test 1**: the code for test 1 can be found in `test/task3_test1.jl`. We decided to advantage of multi-threading for this test, since each simulation can be run independently, and their results can be stored without affecting any other simulations (see the source code for more details). To make use of multi-threading, you have to run Julia with the `-t` or `--threads` option, and specify the number of threads you want to utilise, e.g. `julia -t 8` to run Julia with 8 threads. The test can then by executed using:
```julia
task3_test1(
    [scenario1, scenario2, scenario3, scenario4],  # vector of all scenarios to test
    verbose=false,  # whether to print out progress messages as the simulations are run
    multithreaded=true  # whether to utilise multi-threading (julia needs to be run with
                        # `julia -t n` where n is the number of threads)
)
```

An example output is given below, for `max_time` ranging from 10^3 to 10^5 (beyond 10^5 is unreasonable for scenario 4 - it took around half an hour to run scenario 4 with `max_time = 10^6`). In every scenario, the absolute relative errors seem to decrease as $\rho^*$ gets larger, and also as `max_time` gets larger. As can be seen below the absolute relative error decreases as max_time increases.

<center>
    <img src="img/task3test1.png" width="640" />
</center>

**Test 2**: the code for test 2 can be found in `test/task3_test2.jl`. In order to record the number of arrivals $A_i(t)$, we added an `arrivals` field to the `QueueNetworkState` struct. So at the end of the simulation, the `state` will be populated with the total number of arrivals to each node, and we can access this through `state.arrivals`. However, we need to be able to have a reference to `state` from outside the `sim_net()` function to be able to calculate the simulated mean number of arrivals in the test. One solution is to have `sim_net()` return the mean number of arrivals as well, but this isn't really a clean solution and breaks compatibility with previous tests and scripts that rely on `sim_net()` only returning one float (total mean queue length). A better solution would be to add `state` as a keyword argument to `sim_net()`:

```julia
# src/GeneralizedJacksonSim.jl
function sim_net(net::NetworkParameters;
                 state::State=QueueNetworkState(net), max_time::Int64=10^6,
                 warm_up_time::Int64=10^4, seed::Int64=42)::Float64
    
# src/state.jl
mutable struct QueueNetworkState <: State
    queues::Vector{Int}
    arrivals::Vector{Int}
    num_queues::Int
    net::NetworkParameters

    # Inner constructor for a given scenario's parameters
    function QueueNetworkState(net::NetworkParameters)
        new(zeros(Int64, net.L), zeros(Int64, net.L), net.L, net)
    end
end
```

Now if we pass in a `state` to `sim_net()`, we have a reference to the final state of the simulation when it ends, and we can query `state.arrivals` after running `sim_net()` to get the total number of arrivals to each node. Importantly, this doesn't break compatibility with other uses of `sim_net()`, since state will default to `QueueNetworkState(net)` if there isn't a state passed.

```julia
# test/task3_test2.jl
function task3_test2(scenarios::Vector{NetworkParameters};
                     max_time::Int64=10^5, digits::Int64=3, ρ_star=0.8,
                     c_s_values::Vector{Float64}=[0.1, 0.5, 1.0, 2.0, 4.0])
    for (index, scenario) in enumerate(scenarios)
        println("Scenario $index:")
        # make scenario stable if it is not stable
        if maximum(compute_ρ(scenario)) > 1
            scenario = set_scenario(scenario, ρ_star)
        end

        for c_s in c_s_values
            println("   c_s = $c_s:")
            scenario = @set scenario.c_s = c_s
            # pass in a state to sim_net() so we have a reference to the state to calculate the
            # mean number of arrivals
            state = QueueNetworkState(scenario)
            _ = sim_net(scenario, state=state, max_time=max_time)

            # calculate mean number of arrivals
            mean_num_arrivals = state.arrivals ./ max_time
            theoretical_mean_num_arrivals = (I - scenario.P') \ scenario.α_vector
            sum_of_squares = sum((mean_num_arrivals - theoretical_mean_num_arrivals).^2)

            println("       Simulated average number of arrivals  : " *
                    "$(round.(mean_num_arrivals, digits=digits))")
            println("       Theoretical average number of arrivals: " *
                    "$(round.(theoretical_mean_num_arrivals, digits=digits))")
            println("       Sum of squared differences            : $sum_of_squares")
        end
    end
end
```

Running the test for all 4 scenarios gives
```
Scenario 1:
   c_s = 0.1:
       Simulated average number of arrivals  : [0.498, 0.498, 0.498]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 1.8056600000000162e-5
   c_s = 0.5:
       Simulated average number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 2.012429999999971e-5
   c_s = 1.0:
       Simulated average number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 2.9203200000000215e-5
   c_s = 2.0:
       Simulated average number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 2.7785699999999824e-5
   c_s = 4.0:
       Simulated average number of arrivals  : [0.5, 0.5, 0.5]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 6.911999999999437e-7
Scenario 2:
   c_s = 0.1:
       Simulated average number of arrivals  : [0.709, 0.709, 0.709]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 8.637266938775579e-5
   c_s = 0.5:
       Simulated average number of arrivals  : [0.714, 0.714, 0.714]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 6.71897959183815e-7
   c_s = 1.0:
       Simulated average number of arrivals  : [0.713, 0.713, 0.713]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 7.424412244897982e-6
   c_s = 2.0:
       Simulated average number of arrivals  : [0.72, 0.72, 0.72]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 8.472729795918253e-5
   c_s = 4.0:
       Simulated average number of arrivals  : [0.713, 0.713, 0.713]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 2.516197959183839e-6
Scenario 3:
   c_s = 0.1:
       Simulated average number of arrivals  : [0.801, 0.8, 0.798, 0.799, 0.799]
       Theoretical average number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences            : 9.330100000000268e-6
   c_s = 0.5:
       Simulated average number of arrivals  : [0.801, 0.799, 0.797, 0.797, 0.797]
       Theoretical average number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences            : 2.7853900000001146e-5
   c_s = 1.0:
       Simulated average number of arrivals  : [0.798, 0.797, 0.8, 0.799, 0.8]
       Theoretical average number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences            : 1.4997300000000487e-5
   c_s = 2.0:
       Simulated average number of arrivals  : [0.802, 0.803, 0.801, 0.801, 0.803]
       Theoretical average number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences            : 2.235429999999947e-5
   c_s = 4.0:
       Simulated average number of arrivals  : [0.801, 0.8, 0.801, 0.802, 0.802]
       Theoretical average number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences            : 9.980999999999514e-6
Scenario 4:
   c_s = 0.1:
       Simulated average number of arrivals  : [0.397, 0.404, 0.393, 0.397, 0.397, 0.395, 0.398, 0.422, 0.381, 0.407, 0.394, 0.398, 0.383, 0.364, 0.426, 0.417, 0.411, 0.373, 0.381, 0.37, 0.399, 0.368, 0.404, 0.373, 0.391, 0.398, 0.405, 0.386, 0.41, 0.386, 0.414, 0.408, 0.388, 0.379, 0.372, 0.404, 0.384, 0.373, 0.383, 0.384, 0.418, 0.402, 0.415, 0.38, 0.352, 0.392, 0.371, 0.382, 0.374, 0.385, 0.393, 0.372, 0.385, 0.402, 0.418, 0.381, 0.39, 0.404, 0.392, 0.385, 0.395, 0.358, 0.386, 0.41, 0.377, 0.419, 0.382, 0.372, 0.39, 0.396, 0.393, 0.38, 0.369, 0.429, 0.39, 0.381, 0.374, 0.372, 0.382, 0.4, 0.383, 0.394, 0.418, 0.381, 0.41, 0.411, 0.39, 0.383, 0.373, 0.403, 0.378, 0.402, 0.426, 0.388, 0.421, 0.4, 0.401, 0.403, 0.393, 0.372]
       Theoretical average number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408, 0.393, 0.398, 0.38, 0.363, 0.427, 0.415, 0.412, 0.369, 0.385, 0.37, 0.401, 0.369, 0.402, 0.369, 0.39, 0.396, 0.403, 0.388, 0.408, 0.385, 0.415, 0.406, 0.386, 0.379, 0.371, 0.401, 0.384, 0.375, 0.384, 0.385, 0.417, 0.403, 0.415, 0.379, 0.353, 0.392, 0.373, 0.379, 0.375, 0.385, 0.39, 0.369, 0.387, 0.405, 0.419, 0.381, 0.388, 0.406, 0.388, 0.386, 0.392, 0.359, 0.386, 0.407, 0.378, 0.42, 0.38, 0.372, 0.39, 0.397, 0.393, 0.38, 0.37, 0.431, 0.387, 0.381, 0.376, 0.371, 0.384, 0.398, 0.384, 0.397, 0.419, 0.385, 0.411, 0.413, 0.391, 0.386, 0.372, 0.402, 0.379, 0.403, 0.424, 0.385, 0.419, 0.401, 0.4, 0.406, 0.389, 0.372]
       Sum of squared differences            : 0.0003935084561040076
   c_s = 0.5:
       Simulated average number of arrivals  : [0.398, 0.397, 0.395, 0.397, 0.391, 0.391, 0.391, 0.421, 0.38, 0.408, 0.393, 0.4, 0.379, 0.363, 0.428, 0.414, 0.414, 0.369, 0.386, 0.373, 0.403, 0.368, 0.403, 0.369, 0.39, 0.399, 0.403, 0.39, 0.41, 0.386, 0.418, 0.407, 0.385, 0.379, 0.374, 0.402, 0.383, 0.374, 0.385, 0.386, 0.417, 0.405, 0.415, 0.383, 0.35, 0.392, 0.37, 0.379, 0.376, 0.386, 0.387, 0.37, 0.385, 0.404, 0.419, 0.383, 0.389, 0.406, 0.389, 0.387, 0.392, 0.36, 0.389, 0.406, 0.376, 0.419, 0.382, 0.368, 0.39, 0.396, 0.392, 0.381, 0.369, 0.428, 0.391, 0.381, 0.374, 0.371, 0.385, 0.396, 0.387, 0.396, 0.416, 0.384, 0.412, 0.416, 0.389, 0.384, 0.37, 0.401, 0.379, 0.402, 0.423, 0.383, 0.414, 0.396, 0.396, 0.402, 0.388, 0.373]
       Theoretical average number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408, 0.393, 0.398, 0.38, 0.363, 0.427, 0.415, 0.412, 0.369, 0.385, 0.37, 0.401, 0.369, 0.402, 0.369, 0.39, 0.396, 0.403, 0.388, 0.408, 0.385, 0.415, 0.406, 0.386, 0.379, 0.371, 0.401, 0.384, 0.375, 0.384, 0.385, 0.417, 0.403, 0.415, 0.379, 0.353, 0.392, 0.373, 0.379, 0.375, 0.385, 0.39, 0.369, 0.387, 0.405, 0.419, 0.381, 0.388, 0.406, 0.388, 0.386, 0.392, 0.359, 0.386, 0.407, 0.378, 0.42, 0.38, 0.372, 0.39, 0.397, 0.393, 0.38, 0.37, 0.431, 0.387, 0.381, 0.376, 0.371, 0.384, 0.398, 0.384, 0.397, 0.419, 0.385, 0.411, 0.413, 0.391, 0.386, 0.372, 0.402, 0.379, 0.403, 0.424, 0.385, 0.419, 0.401, 0.4, 0.406, 0.389, 0.372]
       Sum of squared differences            : 0.00038710725810373796
   c_s = 1.0:
       Simulated average number of arrivals  : [0.399, 0.403, 0.395, 0.398, 0.395, 0.392, 0.394, 0.416, 0.381, 0.406, 0.386, 0.396, 0.378, 0.365, 0.425, 0.414, 0.416, 0.366, 0.387, 0.366, 0.402, 0.369, 0.404, 0.372, 0.392, 0.396, 0.404, 0.385, 0.406, 0.386, 0.417, 0.405, 0.381, 0.381, 0.371, 0.404, 0.385, 0.375, 0.385, 0.385, 0.417, 0.404, 0.415, 0.378, 0.354, 0.39, 0.372, 0.378, 0.375, 0.383, 0.387, 0.369, 0.386, 0.403, 0.418, 0.385, 0.385, 0.409, 0.388, 0.387, 0.393, 0.36, 0.386, 0.407, 0.378, 0.418, 0.384, 0.373, 0.392, 0.397, 0.393, 0.378, 0.372, 0.429, 0.384, 0.38, 0.373, 0.369, 0.385, 0.4, 0.382, 0.399, 0.42, 0.385, 0.41, 0.415, 0.392, 0.383, 0.372, 0.404, 0.383, 0.402, 0.424, 0.386, 0.421, 0.397, 0.401, 0.404, 0.39, 0.374]
       Theoretical average number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408, 0.393, 0.398, 0.38, 0.363, 0.427, 0.415, 0.412, 0.369, 0.385, 0.37, 0.401, 0.369, 0.402, 0.369, 0.39, 0.396, 0.403, 0.388, 0.408, 0.385, 0.415, 0.406, 0.386, 0.379, 0.371, 0.401, 0.384, 0.375, 0.384, 0.385, 0.417, 0.403, 0.415, 0.379, 0.353, 0.392, 0.373, 0.379, 0.375, 0.385, 0.39, 0.369, 0.387, 0.405, 0.419, 0.381, 0.388, 0.406, 0.388, 0.386, 0.392, 0.359, 0.386, 0.407, 0.378, 0.42, 0.38, 0.372, 0.39, 0.397, 0.393, 0.38, 0.37, 0.431, 0.387, 0.381, 0.376, 0.371, 0.384, 0.398, 0.384, 0.397, 0.419, 0.385, 0.411, 0.413, 0.391, 0.386, 0.372, 0.402, 0.379, 0.403, 0.424, 0.385, 0.419, 0.401, 0.4, 0.406, 0.389, 0.372]
       Sum of squared differences            : 0.0004263142629795542
   c_s = 2.0:
       Simulated average number of arrivals  : [0.402, 0.399, 0.392, 0.397, 0.397, 0.394, 0.396, 0.417, 0.383, 0.412, 0.393, 0.395, 0.381, 0.361, 0.424, 0.415, 0.413, 0.37, 0.386, 0.37, 0.404, 0.369, 0.398, 0.374, 0.39, 0.394, 0.403, 0.388, 0.411, 0.386, 0.41, 0.405, 0.381, 0.378, 0.367, 0.403, 0.388, 0.373, 0.381, 0.386, 0.417, 0.403, 0.415, 0.38, 0.354, 0.394, 0.373, 0.379, 0.375, 0.385, 0.391, 0.37, 0.387, 0.406, 0.418, 0.385, 0.389, 0.4, 0.385, 0.388, 0.393, 0.358, 0.386, 0.405, 0.377, 0.42, 0.381, 0.374, 0.39, 0.394, 0.393, 0.382, 0.37, 0.43, 0.386, 0.377, 0.38, 0.37, 0.385, 0.395, 0.382, 0.396, 0.42, 0.383, 0.414, 0.411, 0.39, 0.386, 0.374, 0.401, 0.376, 0.404, 0.424, 0.386, 0.42, 0.399, 0.397, 0.404, 0.391, 0.375]
       Theoretical average number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408, 0.393, 0.398, 0.38, 0.363, 0.427, 0.415, 0.412, 0.369, 0.385, 0.37, 0.401, 0.369, 0.402, 0.369, 0.39, 0.396, 0.403, 0.388, 0.408, 0.385, 0.415, 0.406, 0.386, 0.379, 0.371, 0.401, 0.384, 0.375, 0.384, 0.385, 0.417, 0.403, 0.415, 0.379, 0.353, 0.392, 0.373, 0.379, 0.375, 0.385, 0.39, 0.369, 0.387, 0.405, 0.419, 0.381, 0.388, 0.406, 0.388, 0.386, 0.392, 0.359, 0.386, 0.407, 0.378, 0.42, 0.38, 0.372, 0.39, 0.397, 0.393, 0.38, 0.37, 0.431, 0.387, 0.381, 0.376, 0.371, 0.384, 0.398, 0.384, 0.397, 0.419, 0.385, 0.411, 0.413, 0.391, 0.386, 0.372, 0.402, 0.379, 0.403, 0.424, 0.385, 0.419, 0.401, 0.4, 0.406, 0.389, 0.372]
       Sum of squared differences            : 0.0004501958361770971
   c_s = 4.0:
       Simulated average number of arrivals  : [0.397, 0.4, 0.393, 0.398, 0.395, 0.39, 0.392, 0.422, 0.386, 0.407, 0.394, 0.396, 0.378, 0.363, 0.427, 0.412, 0.411, 0.366, 0.382, 0.367, 0.399, 0.369, 0.401, 0.368, 0.389, 0.397, 0.4, 0.385, 0.408, 0.386, 0.413, 0.407, 0.386, 0.378, 0.368, 0.401, 0.388, 0.376, 0.384, 0.384, 0.416, 0.403, 0.412, 0.377, 0.352, 0.392, 0.37, 0.379, 0.373, 0.386, 0.392, 0.37, 0.386, 0.404, 0.421, 0.383, 0.385, 0.409, 0.392, 0.384, 0.391, 0.36, 0.384, 0.411, 0.377, 0.416, 0.378, 0.371, 0.39, 0.397, 0.394, 0.38, 0.367, 0.43, 0.386, 0.379, 0.378, 0.373, 0.387, 0.397, 0.387, 0.394, 0.417, 0.384, 0.413, 0.414, 0.39, 0.383, 0.371, 0.405, 0.381, 0.404, 0.422, 0.389, 0.421, 0.396, 0.401, 0.406, 0.39, 0.375]
       Theoretical average number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408, 0.393, 0.398, 0.38, 0.363, 0.427, 0.415, 0.412, 0.369, 0.385, 0.37, 0.401, 0.369, 0.402, 0.369, 0.39, 0.396, 0.403, 0.388, 0.408, 0.385, 0.415, 0.406, 0.386, 0.379, 0.371, 0.401, 0.384, 0.375, 0.384, 0.385, 0.417, 0.403, 0.415, 0.379, 0.353, 0.392, 0.373, 0.379, 0.375, 0.385, 0.39, 0.369, 0.387, 0.405, 0.419, 0.381, 0.388, 0.406, 0.388, 0.386, 0.392, 0.359, 0.386, 0.407, 0.378, 0.42, 0.38, 0.372, 0.39, 0.397, 0.393, 0.38, 0.37, 0.431, 0.387, 0.381, 0.376, 0.371, 0.384, 0.398, 0.384, 0.397, 0.419, 0.385, 0.411, 0.413, 0.391, 0.386, 0.372, 0.402, 0.379, 0.403, 0.424, 0.385, 0.419, 0.401, 0.4, 0.406, 0.389, 0.372]
       Sum of squared differences            : 0.0004319871998824227

```

## Task 4

For task 4, a similar code to task 2 was implemented. The full file for task for can be located in task4.jl. The code below generates simulated man queue length for each service time at each p* value. The function takes a specific network (i.e. scenario 1-4) so varing data can be generated. The array, plotting_data, stores 5 vectors, one for each service time.

In [ ]:
using Pkg, Parameters, LinearAlgebra, Plots, Statistics, StatsBase
Pkg.activate(".")

include("../simulation_script.jl")

include("../test/scenarios.jl")

service_time_values = [0.1,0.5,1.0,2.0,4.0]
scenarios = [scenario1, scenario2, scenario3, scenario4]

#generates the y-values for each service time for each scenario
function plot_mean_queue_length_service_times(net::NetworkParameters, scenario_number::Int64; max_time=10^3, warm_up_time=10)
    ρ_star_values = 0.01:0.01:0.9
    service_time_values = [0.1,0.5,1.0,2.0,4.0]
    plotting_data = Vector{Vector{Float64}}()
#forming data for each service time
    for (j,service_time) in enumerate(service_time_values)
        simulated_total_mean_queue_lengths = zeros(length(ρ_star_values))
        #forming data for each p*
        for (index, ρ_star) in enumerate(ρ_star_values)
            # adjust network parameters
            adjusted_net = set_scenario(net, ρ_star, service_time)
            simulated = sim_net(adjusted_net, max_time=max_time, warm_up_time=warm_up_time)
            # get total steady state mean queue length
            simulated_total_mean_queue_lengths[index] = simulated
        end
        #pushes vector of mean queue lengths for each service time into plotting_data vector
        push!(plotting_data, simulated_total_mean_queue_lengths)
    end
    return plotting_data
end


: 

To plot the required graphs the above function is called in the below function plotting_service_times, which allows the 5 varying service times for each scenario to be plotted on one figure, by extracting data from the plotting_data array.

In [ ]:
#plots graphs for each scenario with varying service times displayed
function plotting_service_times(net::NetworkParameters, scenario_number::Int64)
    ρ_star_values = 0.01:0.01:0.9
    data = plot_mean_queue_length_service_times(net, scenario_number,max_time=10^3, warm_up_time=10)
    return plot(ρ_star_values,[data[1],data[2],data[3],data[4],data[5]], xlabel="ρ",
        ylabel="Total Mean Queue Length",
        title="Scenario $scenario_number Simulation with varying service times",
        labels=["0.1" "0.5" "1.0" "2.0" "4.0"])
end

#plotting initial servcice time graphs
p1 = plotting_service_times(scenario1, 1,)
p2 = plotting_service_times(scenario2, 2,)
p3 = plotting_service_times(scenario3, 3,)
p4 = plotting_service_times(scenario4, 4,)
return plot(p1,p2,p3,p4, layout=(4, 1), legend=true, size=(900, 900))

: 

This results in the below graphs, it can be seen that higher service times e.g. 4.0 result in a mean total queue length compared to lower service times e.g. 0.1

<center>
    <img src="img/task4_service_times.png" width="640" />
</center>

Confidence bounds for these curves were also generated using the code below, where 100 different points were generated for each p* by using a different seed for each generation as can be seen in the for loops below.

In [ ]:
#calculates and returns plot of confidence bounds for each scenario and service time
function confidence_bounds(net::NetworkParameters, scenario_number::Int64, service_time::Float64,max_time=10^3, warm_up_time=10)
    ρ_star_values = 0.01:0.01:0.9
    data = []
    mean_data=[]
    top_percentile = []
    bottom_percentile = []
        for (index, ρ_star) in enumerate(ρ_star_values)
            adjusted_net = set_scenario(net, ρ_star, service_time)
            #generates simulated data for each p* 100 times at 100 varying seeds
            for seed in 1:100
                simulated = sim_net(adjusted_net, max_time=max_time, warm_up_time=warm_up_time,seed = seed)
                push!(data,simulated)
            end
        #pushes in the mean and upper and lower confidence bounds data
        push!(mean_data, mean(data))
        push!(top_percentile, percentile(data,95))
        push!(bottom_percentile, percentile(data,5))
        end
#plots confidence bounds graph
return plot(ρ_star_values,mean_data; ribbon = (bottom_percentile,top_percentile), xlabel="ρ",
    ylabel="Total Mean Queue Length",
    title="Scenario $scenario_number Simulation with $service_time service time, confidence bound",
    labels=["mean" "95th percentile" "5th percentile"])
end

: 

This function could then be called for each scenario and service time using code of the format below: 

In [ ]:
plots = Vector()
for (index,scenario) in enumerate(scenarios)
  for service_time in service_time_values
    push!(plots, confidence_bounds(scenario, index, service_time))
  end
end
return plot(plots..., layout=(19, 1), legend=true, size=(1100, 1100))

: 

The resulting confidence bounds for each scenario were as below:

<center>
    <img src="img/scenario1_confbounds.png" width="640" />
</center>

<center>
    <img src="img/scenario2_confbounds.png" width="640" />
</center>

<center>
    <img src="img/scenario3_confbound.png" width="640" />
</center>

<center>
    <img src="img/scenario4_confbounds.png" width="640" />
</center>




## Task 5

`sim_net_customers()` is largely the same as `sim_net()`, except for a few changes. The state used is a `CustomerQueueNetworkState`, which implements `queues` as a vector of `Queue{Customer}`s instead of a vector of `Int64`s, and keeps track of all departed customers in the field `departed_customers`. That is, we keep track of the individual customers rather than just the number of customers in each queue.

```julia
# src/customer.jl
mutable struct Customer
    arrival::Float64
    departure::Float64
end

# src/state.jl
mutable struct CustomerQueueNetworkState <: State
    queues::Vector{Queue{Customer}}
    departed_customers::Vector{Customer}
    arrivals::Vector{Int64}
    num_queues::Int64
    net::NetworkParameters

    # Inner constructor for a given scenario's parameters
    function CustomerQueueNetworkState(net::NetworkParameters)
        queues = Vector{Queue{Customer}}()
        for _ in 1:net.L
            push!(queues, Queue{Customer}())
        end
        new(queues, Vector{Customer}(), zeros(Int64, net.L), net.L, net)
    end
end
```

`record_integral()` in `sim_net_customers()` also needs to be changed; we need to map the vector of `Queue{Customer}`s to a vector of the length of each queue:

```julia
# src/GeneralizedJacksonSim.jl
function record_integral(time::Float64, state::State)
    if time >= warm_up_time
        queues_integral += (map((queue) -> length(queue), state.queues) *
            (time - last_time))
    end
    last_time = time
end
```

The biggest changes are in the `process_event()` functions. We introduce the following new `Event` types for `Customer`s:

```julia
# src/event.jl
struct CustomerExternalArrivalEvent <: Event
    q::Int64
end
struct CustomerEndOfServiceEvent <: Event
    q::Int64
end
```

On arrival into the system, a `Customer` is created with the corresponding `arrival` time, and `-1.0` as the `departure` time (`-1.0` signifying that the customer hasn't left the system yet).
```julia
# src/event.jl
function process_event(time::Float64, state::CustomerQueueNetworkState,
                       event::CustomerExternalArrivalEvent)
    q = event.q
    enqueue!(state.queues[q], Customer(time, -1.0))  # add new customer to queue
    state.arrivals[q] += 1  # record arrival
    new_timed_events = TimedEvent[]

    # prepare next external arrival for this particular server
    push!(new_timed_events,
        TimedEvent(CustomerExternalArrivalEvent(q), time + next_arrival_time(state, q)))

    # start serving this job if it is the only one in the queue
    if length(state.queues[q]) == 1
        push!(new_timed_events,
            TimedEvent(CustomerEndOfServiceEvent(q), time + next_service_time(state, q)))
    end
    return new_timed_events
end
```

When a customer finishes at a server, we record when it leaves the system in the `customer`'s `departure` field, and add the customer to the `state`'s `departed_customers` field. If the customer is just moving to another server, the logic is the same as for an `EndOfServiceEvent`.

```julia
# src/event.jl
function process_event(time::Float64, state::CustomerQueueNetworkState,
                       event::CustomerEndOfServiceEvent)
    q = event.q
    customer = dequeue!(state.queues[q])  # remove from queue
    @assert length(state.queues[q]) >= 0
    new_timed_events = TimedEvent[]

    # if there is another customer in the queue, start serving them
    if length(state.queues[q]) > 0
        push!(new_timed_events,
            TimedEvent(CustomerEndOfServiceEvent(q), time + next_service_time(state, q)))
    end

    # simulate the next location for this job; indices 1:L are the probabilities of moving
    # to another server in the system, and the last index is the probability of exiting
    # the system
    L = state.net.L
    next_loc_weights = state.net.P[q, :]
    push!(next_loc_weights, 1 - sum(next_loc_weights))
    @assert sum(next_loc_weights) == 1
    next_loc = sample(1:L+1, Weights(next_loc_weights))

    if next_loc <= L
        enqueue!(state.queues[next_loc], customer)  # job is staying in the system
        state.arrivals[next_loc] += 1  # record arrival

        # start serving job if it is the only one in the queue
        if length(state.queues[next_loc]) == 1
            service_time = next_service_time(state, next_loc)
            push!(new_timed_events,
                TimedEvent(CustomerEndOfServiceEvent(next_loc), time + service_time))
        end
    else # job leaving the system 
        customer.departure = time
        push!(state.departed_customers, customer)
    end
    return new_timed_events
end
```

We can use the `state`'s `departed_customers` vector to estimate the Q1, median, and Q3 sojourn time like so:

In [ ]:
function estimate_sojourn_time(scenarios::Vector{NetworkParameters}; max_time::Int64=10^6,
                               warm_up_time::Int64=10^4, ρ_star::Float64=0.8,
                               c_s_values::Vector{Float64}=[0.5, 1.0, 2.0])
    for (index, scenario) in enumerate(scenarios)
        println("Scenario $index:")
        scenario = set_scenario(scenario, ρ_star)
        for c_s in c_s_values
            println("    c_s = $c_s:")
            scenario = @set scenario.c_s = c_s
            state = CustomerQueueNetworkState(scenario)
            sim_net_customers(scenario, state=state, max_time=max_time, warm_up_time=warm_up_time)

            sojourn_times = map((c) -> (c.departure - c.arrival), state.departed_customers)
            quartiles = nquantile(sojourn_times, 4)
            println("        Estimated Q1: $(quartiles[2])")
            println("        Estimated Q2: $(quartiles[3])")
            println("        Estimated Q3: $(quartiles[4])")
        end
    end
end

scenarios = [scenario1, scenario2, scenario3, scenario4]
estimate_sojourn_time(scenarios, max_time=10^5, warm_up_time=10^3)

: 

In a formatted table:

| Scenario | c_s value | Estimated Q1 | Estimated Q2 | Estimated Q3 |
| -------- | --------- | ------------ | ------------ | ------------ |
| Scenario 1 | 0.5 | 6.148440414075594 | 9.16471545411514 | 13.35092333636203 |
| Scenario 1 | 1.0 | 8.607981412327717 | 13.36314495741317 | 19.61502679703335 |
| Scenario 1 | 2.0 | 13.465573404418137 | 20.958324511955652 | 31.15071603749641 |
| Scenario 2 | 0.5 | 7.321437295458054 | 11.776672688079998 | 19.02253944581753 |
| Scenario 2 | 1.0 | 10.210841941421677 | 16.59163452223038 | 26.39844796482066 |
| Scenario 2 | 2.0 | 16.085610787142286 | 25.872199326993723 | 41.19310562303872 |
| Scenario 3 | 0.5 | 0.9152987968991511 | 3.3518831798610336 | 7.734204929642601 |
| Scenario 3 | 1.0 | 0.951647432105915 | 3.730264990048454 | 8.924551431995496 |
| Scenario 3 | 2.0 | 0.987596887318432 | 4.677023068732524 | 12.204979815089246 |
| Scenario 4 | 0.5 | 1.2585334678306026 | 2.8046149461442837 | 5.81824992953625 |
| Scenario 4 | 1.0 | 1.1594554236075965 | 3.0126887688547868 | 6.698496126526152 |
| Scenario 4 | 2.0 | 0.977942439149956 | 3.3644763502179558 | 8.408937287655135 |

: 